<a href="https://colab.research.google.com/github/drbarak/NBA/blob/main/NCAA2019.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# see https://stackoverflow.com/questions/52292456/installing-jupyter-contrib-nbextension-on-google-colab
#!pip install jupyter_contrib_nbextensions
#!jupyter contrib nbextension install --user
#!jupyter nbextension enable freeze/main

In [46]:
# General tools
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from IPython.display import display
p = print
d = display

In [47]:
import sys
#sys.path.append("D:/Dropbox/DataScience")
#from my_utilities import my_utils as my

In [48]:
%%HTML
<style type="text/css">
table.dataframe td, table.dataframe th {
    border: 1px  black solid !important;
  color: black !important;
}
</style>

In [49]:
import pickle

def save_obj(obj, name ):
    with open(name + '.pkl', 'wb+') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

def load_obj(name ):
    with open(name + '.pkl', 'rb') as f:
        return pickle.load(f)

In [50]:
pd.set_option("precision", 2)
ncaa_dir = '/content/drive/MyDrive/NCAA/2021/MDataFiles_Stage1/'  # so can easily modified later

https://www.kaggle.com/c/ncaam-march-mania-2021

> based on blog: https://www.kdnuggets.com/2017/03/machine-learning-march-madness.html

> notebook: https://github.com/adeshpande3/March-Madness-2017/blob/master/March%20Madness%202017.ipynb

# Data

In [51]:
year = 2019
#year = 2018
#year = 2017
#year = 2016

# changes of columns names from 2016-2017 to 2021
Team = 'TeamID'
Team_Name = 'TeamName'
Team_Id = 'TeamID'
Wteam = 'WTeamID'
Wscore = 'WScore'
Lteam = 'LTeamID'
Lscore = 'LScore'
Wloc = 'WLoc'
Numot = 'NumOT'
Wfgm3 = 'LFGM3'
Lfgm3 = 'WFGM3'
DayNum = 'DayNum'

In [52]:
def get_data(fname):
  teams_df = pd.read_csv(ncaa_dir + fname + '.csv')
  p(fname + '_df', teams_df.shape)
  d(teams_df.head())
  return teams_df

## 1. Teams

FirstD1Season: Team first participation year

LastD1Season: Team last participation year

Because March Madness games are for Division-I game, each team might appear/disappear in each year.

In [ ]:
teams_df = get_data('MTeams')

MTeams_df (371, 4)


,TeamID,TeamName,FirstD1Season,LastD1Season
0,1101,Abilene Chr,2014,2021
1,1102,Air Force,1985,2021
2,1103,Akron,1985,2021
3,1104,Alabama,1985,2021
4,1105,Alabama A&M,2000,2021


In [54]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
teams_df[teams_df[Team_Name].str.find('Okl') >= 0]

## 2. Seasons

DayZero: All game dates have been aligned upon a common scale so that (each year) the Monday championship game of the men's tournament is on DayNum=154.

RegionW, RegionX, Region Y, Region Z: Tournament proceeds with dividing US in 4 regions, but this division changes in year. These RegionW, X, Y, Z column represents 4 regions of each year. Note that RegionW & X, Y & Z have matchup in semifinals. Winner of WX and YZ regions have the final championship matchup.

In [ ]:
seasons_df = get_data('MSeasons')

MSeasons_df (37, 6)


,Season,DayZero,RegionW,RegionX,RegionY,RegionZ
0,1985,10/29/1984,East,West,Midwest,Southeast
1,1986,10/28/1985,East,Midwest,Southeast,West
2,1987,10/27/1986,East,Southeast,Midwest,West
3,1988,11/2/1987,East,Midwest,Southeast,West
4,1989,10/31/1988,East,West,Midwest,Southeast


## 3. Seeds

Seed column string consists of Region (W, X, Y, Z) + seed number (01 ~ 16). Seed number 01 is considered as most "strong". For play-in games, which occurs before main tournament, fourth character (a or b) is added.

Before tournament, 68 teams will be reduced to 64 teams, so 4 teams have pre-tournament match up. "a" and "b" is the match up of those teams.

In [ ]:
tourneySeeds_df = get_data('MNCAATourneySeeds')

In [ ]:
tourneySeeds_df.tail()

In [ ]:
team = 1104  # alabama
team = 1328  # Oklahoma
team = 1181  # 

mask = (tourneySeeds_df.Season == year) & (tourneySeeds_df[Team] == team)
tourneySeeds_df[mask].Seed.str[1:]

## 4. Regular Season Results

For each year, we have regular season at first, and only 68 teams survive for NCAA tournament, also called March Madness (In parallel, some teams who cannot proceed March Madness participates Secondary tournament). The game result of these 3 are stored in separated files.

"Compact results" data stores important information for win/lose, while "Detailed results" data (explained later) stores much precise information. Let's see Compact results data here.

Store matchup date, winner/loser's team id & score.

Wteam: id of winning team
Lteam: id of loosing team
WScore: is the winner score at the end of game
LScore: is the loser score at the end of game
WLoc: the "location" of winning team, either ("H", "A", "N") - "H" is home, "A" is Away (visiting to opponent's site), "N" is neutral court.
NumOT: the number of overtime periods (when the score is same after 4-period of game, we continue the game with overtime period in basketball.)

The 4 columns, Season, DayNum, WTeamID and LTeamID uniquely identifies the game. This fact is very important when you want to merge information of other files.

In [55]:
regularSeasonCompactResults_df = get_data('MRegularSeasonCompactResults')

MRegularSeasonCompactResults_df (166880, 8)


,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT
0,1985,20,1228,81,1328,64,N,0
1,1985,25,1106,77,1354,70,H,0
2,1985,25,1112,63,1223,56,H,0
3,1985,25,1165,70,1432,54,H,0
4,1985,25,1192,86,1447,74,H,0


### Detailed

This section provides game-by-game stats at a team level (free throws attempted, defensive rebounds, turnovers, etc.) for all regular season, conference tournament, and NCAA® tournament games since the 2002-03 season.

Team Box Scores are provided in "Detailed Results" files rather than "Compact Results" files.

Compact results are collected from 1985, but details results are collected since 2003, so the number of rows in the data are different.

But 8 columns (Season, DayNum, WTeamID, WScore, LTeamID, LScore, WLoc, and NumOT) are exactly same value with CompactResults file.

Additional columns stores more precise information:

WFGM - field goals made (by the winning team)
WFGA - field goals attempted (by the winning team)
WFGM3 - three pointers made (by the winning team)
WFGA3 - three pointers attempted (by the winning team)
WFTM - free throws made (by the winning team)
WFTA - free throws attempted (by the winning team)
WOR - offensive rebounds (pulled by the winning team)
WDR - defensive rebounds (pulled by the winning team)
WAst - assists (by the winning team)
WTO - turnovers committed (by the winning team)
WStl - steals (accomplished by the winning team)
WBlk - blocks (accomplished by the winning team)
WPF - personal fouls committed (by the winning team)

same set of stats from the perspective of the losing team starting with "L".

For example in top row WFGM=27, WFGM3=3, WFTM=11 and WScore=68.

This means (27-3)=24 goals with 2 points shot, 3 goals with 3 points shot, and 11 goals with 1 point free throw shot.
In total 24 * 2 + 3 * 3 + 11 * 1 = 68.

In [56]:
regularSeasonDetailedResults_df = get_data('MRegularSeasonDetailedResults')

MRegularSeasonDetailedResults_df (92832, 34)


,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,WFGM,WFGA,WFGM3,WFGA3,WFTM,WFTA,WOR,WDR,WAst,WTO,WStl,WBlk,WPF,LFGM,LFGA,LFGM3,LFGA3,LFTM,LFTA,LOR,LDR,LAst,LTO,LStl,LBlk,LPF
0,2003,10,1104,68,1328,62,N,0,27,58,3,14,11,18,14,24,13,23,7,1,22,22,53,2,10,16,22,10,22,8,18,9,2,20
1,2003,10,1272,70,1393,63,N,0,26,62,8,20,10,19,15,28,16,13,4,4,18,24,67,6,24,9,20,20,25,7,12,8,6,16
2,2003,11,1266,73,1437,61,N,0,24,58,8,18,17,29,17,26,15,10,5,2,25,22,73,3,26,14,23,31,22,9,12,2,5,23
3,2003,11,1296,56,1457,50,N,0,18,38,3,9,17,31,6,19,11,12,14,2,18,18,49,6,22,8,15,17,20,9,19,4,3,23
4,2003,11,1400,77,1208,71,N,0,30,61,6,14,11,13,17,22,12,14,4,4,20,24,62,6,16,17,27,21,15,12,10,7,1,14


In [ ]:
#get # wins of a team for a given year
df = regularSeasonDetailedResults_df
# data for each game in a given year
df_year = df[df.Season == year]
# data for the team in that year
df_team = df_year[(df_year[Wteam] == team) | (df_year[Lteam] == team)]
numGames = len(df_team)

# data for games won/lost in that year
df_games_won = df_team[df_team[Wteam] == team]
numWins = len(df_games_won)
totPtsWin = df_games_won[Wscore].sum()
avgPtsWin = totPtsWin / numWins
tot3ptsWin = df_games_won[Wfgm3].sum()
avg3ptsWin = tot3ptsWin / numWins
winDays = df_games_won[DayNum].sum()
p(winDays / numWins)

df_games_lost = df_team[df_team[Lteam] == team]
numLoses = len(df_games_lost)
totPtsLoss = df_games_lost[Lscore].sum()
avgPtsLoss = totPtsLoss / numLoses
avg3ptsLoss = df_games_lost[Lfgm3].mean()

avgPtsGame= (totPtsWin + totPtsLoss) / numGames 
totTotPtsGame = totPtsWin + totPtsLoss
avgTotPtsGame = totTotPtsGame / numGames
avg3ptsGame = (df_team[Wfgm3] + df_team[Wfgm3]).mean()

format_ =  ':\t{} {} {:0.2f}'
p('Tot'+ format_.format(numGames, avgPtsGame, avgTotPtsGame))
p('Win'+ format_.format(numWins, totPtsWin, avgPtsWin))
p('Loss'+ format_.format(numLoses, totPtsLoss, avgPtsLoss))

In [ ]:
df_team.columns

In [ ]:
tourneySeeds_df[tourneySeeds_df.Season == year]

In [ ]:
mask = (tourneySeeds_df.Season == year) & (tourneySeeds_df[Team] == team)
seed = 0 if len(tourneySeeds_df[mask].Seed) == 0 else int(tourneySeeds_df[mask].Seed.str[1:3])
p(seed)
#games_df['Seed'] = seed 

## 5. Tournament Results

Same format as regular season file
For tournament all games are neutral site (so WLoc is always N).

In [ ]:
tourneyCompactResults_df = get_data('MNCAATourneyCompactResults')

In [ ]:
tourneyCompactResults_df.tail()

In [ ]:
tourneyDetailedResults_df = get_data('MNCAATourneyDetailedResults')

In [ ]:
tourneyDetailedResults_df[(tourneyDetailedResults_df.Season==year) & ((tourneyDetailedResults_df[Wteam] == team) | (tourneyDetailedResults_df[Lteam] == team))]

In [ ]:
#games_df['lastTourney'] = 1 if len(tourneyDetailedResults_df[(tourneyDetailedResults_df.Season==year) & ((tourneyDetailedResults_df[Wteam] == team) | (tourneyDetailedResults_df[Lteam] == team))]) > 0 else 0

In [ ]:
#games_df['lastTourneyWins'] = len(tourneyDetailedResults_df[(tourneyDetailedResults_df.Season==year) & (tourneyDetailedResults_df[Wteam] == team)])

In [ ]:
#games_df['prevTourneyWins'] = len(tourneyDetailedResults_df[(tourneyDetailedResults_df.Season < year) & (tourneyDetailedResults_df[Wteam] == team)])

In [ ]:
#games_df

## 6. Tournament Slots

NCAA tournament is organized that it has consistent tournament structure in each year.
Below "slots" information indicates which part of the tournament bracket structure each seed team in.

"GameSlot" shows "most strongest seed slot" so when W11 team wins to W6 in the previous round, this team's game slot is recorded as W6 in the later round.

The tournament slot can be understood by the figure provided in official NCAA website, which is introduced in this kernel by @headsortails.
First Round=1, Second Round=2, Regional Semifinal=3, Regional Final=4, National Semifinal=5, 6=National championship

In [ ]:
tourneySlots_df = get_data('MNCAATourneySlots')

![image.png](attachment:f4d10d59-2029-45f8-84ee-4737a42a31a9.png)

For example, Round 1 of Region W in Season A consists of following 8 matchups.

In [ ]:
tourneySlots_df[(tourneySlots_df['Season'] == 1985) & (tourneySlots_df['Slot'].str.startswith('R1W'))]

## 7. Sample submission

In [ ]:
sampleSubmission = get_data('MSampleSubmissionStage1')


# Prepare season data

In [ ]:
  def get_seed(year, team):
    ## prev year turney
    mask = (tourneySeeds_df.Season == year - 1) & (tourneySeeds_df[Team] == team)
    df_seed = tourneySeeds_df[mask]
    seed = 0 if len(df_seed) == 0 else int(df_seed.Seed.str[1:3])
    return seed

In [ ]:
SEED_FACTOR=10 # so that 0 has much lower ranking than teams that were in the torney
def getSeasonData(team, year):
  # data for each game in a given year
  df_year = regularSeasonDetailedResults_df[regularSeasonDetailedResults_df.Season == year]
  df_year_prev = regularSeasonDetailedResults_df[regularSeasonDetailedResults_df.Season == year - 1]
  # data for the team in that year
  df_team = df_year[(df_year[Wteam] == team) | (df_year[Lteam] == team)]
  df_team_prev = df_year_prev[(df_year_prev[Wteam] == team) | (df_year_prev[Lteam] == team)]

  # data for games won/lost in that year
  df_games_won = df_team[df_team[Wteam] == team]
  df_games_lost = df_team[df_team[Lteam] == team]
  df_games_won_prev = df_team_prev[df_team_prev[Wteam] == team]
  df_games_lost_prev = df_team_prev[df_team_prev[Lteam] == team]

  numWins = len(df_games_won)
  numWins_prev = len(df_games_won_prev)
  wins = {'Wins': numWins, 'Wins_prev': numWins_prev, 'Wseed': 0}
  for col in ['WScore', 'Wallow', 'WFGM', 'WFGA', 'WFGM3', 'WFGA3', 'WFTM', 'WFTA', 'WOR', 'WDR',
         'WAst', 'WTO', 'WStl', 'WBlk', 'WPF', 'DayNum']:
    col_ = col if col != 'Wallow' else 'LScore'
    colTot = df_games_won[col_].sum()
    colTot += df_games_won_prev[col_].sum()
    wins[col] = (colTot, colTot / (numWins + numWins_prev))
  for idx, row in df_games_won.iterrows():
    wins['Wseed'] += get_seed(year, row[Lteam])
  for idx, row in df_games_won_prev.iterrows():
    wins['Wseed'] += get_seed(year, row[Lteam])
#  p(wins)
  numLosses = len(df_games_lost)
  numLosses_prev = len(df_games_lost_prev) 
  losses = {'Losses': numLosses, 'Losses_prev': numLosses_prev, 'Lseed': 0}
  for col in ['LScore', 'Lallow', 'LFGM', 'LFGA', 'LFGM3', 'LFGA3',
         'LFTM', 'LFTA', 'LOR', 'LDR', 'LAst', 'LTO', 'LStl', 'LBlk', 'LPF', 'DayNum']:
    col_ = col if col != 'Lallow' else 'WScore'
    colTot = df_games_lost[col_].sum()
    colTot += df_games_lost_prev[col_].sum()
    losses[col] = (colTot, colTot / (numLosses + numLosses_prev))
  for idx, row in df_games_lost.iterrows():
    losses['Lseed'] += get_seed(year, row[Wteam])
  for idx, row in df_games_lost_prev.iterrows():
    losses['Lseed'] += get_seed(year, row[Wteam])
#  p(losses)
  numGames = numWins + numLoses
  games = {'Games': numGames, 'Gseed': 0}
  for col in ['LScore', 'Lallow', 'LFGM', 'LFGA', 'LFGM3', 'LFGA3',
         'LFTM', 'LFTA', 'LOR', 'LDR', 'LAst', 'LTO', 'LStl', 'LBlk', 'LPF']:
    col_ = col[1:]
    colTot = wins['W' + col_][0] + losses[col][0]
    colAvg = colTot / numGames 
    games['G' + col_] = (colTot, colAvg)
  games['GseedW'] = wins['Wseed']
  games['GseedL'] = -losses['Lseed']
#  games['WDayNum'] = wins['DayNum']
#  games['LDayNum'] = losses['DayNum']
#  p(games)
  games_df = pd.DataFrame(games)
  games_df.drop(0, axis=0, inplace=True)
#  games_df['Team'] = team
  games_df['Games'] = numWins

  seed = get_seed(year, team)
  seed = 0 if seed == 0 else seed + SEED_FACTOR # so that 0 has much lower ranking than teams that were in the tourney
  games_df['Seed'] = seed
  df = tourneyDetailedResults_df
  games_df['lastTourney'] = 1 if len(df[(df.Season==year) & ((df[Wteam] == team) | (df[Lteam] == team))]) > 0 else 0
  games_df['lastTourneyWins'] = len(df[(df.Season==year) & (df[Wteam] == team)])
  games_df['prevTourneyWins'] = len(df[(df.Season < year) & (df[Wteam] == team)])
  games_df['lastTourneyLosses'] = -len(df[(df.Season==year) & (df[Lteam] == team)])
  games_df['prevTourneyLosses'] = -len(df[(df.Season < year) & (df[Lteam] == team)])
  
  return games_df

In [ ]:
#1401 2015 0       False
tourneySeeds_df[(tourneySeeds_df.Season == year-1) & (tourneySeeds_df[Team] == 1401)]

In [ ]:
# test season vector for a team = Oklahoma
getSeasonData(1328, year)

In [ ]:
# test season vector for a team = Oklahoma
getSeasonData(1328, year)

In [ ]:
teams_df[teams_df[Team_Name].str.find('Kentu') >= 0]

In [ ]:
# test season vector for another team
getSeasonData(1246, year)

## Compare 2 teams

In [ ]:
def compareTwoTeams(team1, team2, year):
    df1 = getSeasonData(team1, year)
    df2 = getSeasonData(team2, year)
    return df1 - df2

In [ ]:
compareTwoTeams(1328, 1246, year)

## List of teams in 2019 tourney

In [ ]:
def team_list():
  df = tourneyCompactResults_df
  sw = set(df[(df.Season==year)][Wteam])
  sl = set(df[(df.Season==year)][Lteam])
  teams_list = list(sw | sl)
  len(teams_list)

In [ ]:
def compare_dict():
  eq = True
  for k, v in t_vectors.items():
    if k in teams_vectors:
      v1 = teams_vectors[k]
      if v1.equals(v):
        continue
      eq = False
      break
    else:
      eq = False
      break
  eq

# Prepare X

> We use the list of all the the games in the regular season and the tourney of the year before

> for each game, we calc the diff between the 2 vectors and add the Home feature (using the Wloc column) and the y (win or loss)

> We also create Loss records - to have **Negative Sampling**

In [ ]:
try:
  del teams_vectors
except:
  pass
teams_vectors={}

season = regularSeasonCompactResults_df[(regularSeasonCompactResults_df.Season == year)]
tourney = tourneyCompactResults_df[tourneyCompactResults_df.Season == year - 1]

def create_X():
  X = pd.DataFrame()
  count = 0
  
  for arr in [season, tourney]:
    for index, row in arr.iterrows():
      teamW = row[Wteam]
      teamL = row[Lteam] 

      if not teamW in teams_vectors:
        teams_vectors[teamW] = getSeasonData(teamW, year)
      dfw = teams_vectors[teamW]

      if not teamL in teams_vectors:
        teams_vectors[teamL] = getSeasonData(teamL, year)
      dfl = teams_vectors[teamL]

      diff = dfw - dfl
      diff['Wloc'] = 1 if row[Wloc] == 'H' else 0 if row[Wloc] == 'N' else -1
      diff['Win'] = 1
      X = X.append(diff)
      
      diff = -diff
      diff['Win'] = 0
      X = X.append(diff)

      '''
      if count % 2 == 0:
        diff['Win'] = 1
      else:
        diff['Win'] = 0
        diff = -diff
      '''
      count += 1
#  X.head()
#  X.to_csv('NCAA2019.csv', index = False)
#  save_obj(teams_vectors, 'teams_vectors')
  return X

In [ ]:
p(len(teams_vectors))
if False and len(teams_vectors) == 0:   
  try:
    teams_vectors = load_obj('teams_vectors2019')
    X_full = pd.read_csv('NCAA2019.csv', usecols=range(1,23))
  except:
    X_full = create_X()

X_full = create_X()
    
p(len(teams_vectors))    
p(X_full.shape)
X_full.columns

# Models

In [ ]:
from sklearn.preprocessing import StandardScaler, MaxAbsScaler, Binarizer
from sklearn.preprocessing import FunctionTransformer

#X = pd.DataFrame(StandardScaler().fit_transform(X))

In [ ]:
from sklearn.model_selection import train_test_split

X = X_full.drop('Win', axis=1)
y = X_full.Win.astype(int)

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7, random_state=0)

In [ ]:
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
import math

pd.set_option("precision", 2)
df_result = pd.DataFrame()

In [ ]:
def report(clf, X, y, data_type='Test'):
  global df_result
  name = type(clf).__name__ + ': ' + data_type
  acc = accuracy_score(y_true=y, y_pred=clf.predict(X))
  index = [(name, clf.classes_[0]), (name, clf.classes_[1])]
  cm = pd.DataFrame(confusion_matrix(y_true=y, y_pred=clf.predict(X)),
                    index=clf.classes_,
                    columns=clf.classes_)
  cm.reset_index(drop=True, inplace=True)
  rep = classification_report(y_true=y, y_pred=clf.predict(X), zero_division=1, output_dict=True)
  rep =  pd.DataFrame(rep)
  rep.drop(['accuracy', 'macro avg', 'weighted avg'], axis=1, inplace=True)
  rep = rep.T
  rep['accuracy'] = acc
  rep.support = rep.support.astype(int)
  cols = list(rep.columns[:3]) + ['accuracy', 'support']
  rep = rep[cols]
  rep['precision_avg'] = rep.precision.mean()
  rep['recall_avg'] = rep.recall.mean()
  rep['f1-score_avg'] = rep['f1-score'].mean()
  rep.reset_index(drop=True, inplace=True)
  df = cm.join(rep)
  df['name'] = name
  df.reset_index(inplace=True)
  #df.set_index(['name', 'index'], inplace=True)
  df_result.reset_index(inplace=True)
  df_result = df_result.append(df, ignore_index=True)
  df_result.set_index(['name', 'index'], inplace=True)
  #d(df_result)
  return

In [ ]:
def showFeatureImportance(model, X_train=X_train, threshold=0):
  if type(model).__name__ == 'LogisticRegression':  # no features importance data
    w0 = model.intercept_[0]
    w = model.coef_[0]
    fx_imp = pd.Series(w, index=X_train.columns)
  else:
    fx_imp = pd.Series(model.feature_importances_, index=X_train.columns)
  fx_imp /= fx_imp.sum()
  fx_imp = fx_imp.sort_values()
  fx_imp = fx_imp[fx_imp > threshold]
  fig = plt.figure(figsize=(5,5))
  plt.xlim(0, 1)
  fx_imp.plot(kind='barh', title=type(model).__name__ )

In [ ]:
def process_model(model, X_train=X_train, y_train=y_train, X_test=X_test, y_test=y_test, plot=False, threshold=0):
  p('running: ', type(model).__name__)
  model.fit(X_train, y_train)
  report(model, X_train, y_train, 'Train')
  report(model, X_test, y_test)
  if plot:
    showFeatureImportance(model, X_train, threshold) 

In [ ]:
X.head()

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier

df_result = pd.DataFrame()

clf_LR = LogisticRegression(C=10, fit_intercept=False, max_iter=1000, warm_start=True)
process_model(clf_LR, threshold=0.01, plot=False)
clf_RF = RandomForestClassifier(n_estimators=500, min_samples_split=6, criterion='entropy')
#process_model(clf_RF, threshold=0.01, plot=True)
clf_GB = GradientBoostingClassifier(n_estimators=100)
#process_model(clf_GB, threshold=0.01, plot=True)

d(df_result)

# Predict a game

In [ ]:
def predictGame(team1, team2, home=0):
  diff = teams_vectors[team1] - teams_vectors[team2]
  diff['Wloc'] = home
  d(diff)
  p(model.predict(diff)[0])
  p(model.predict_proba(diff))

In [ ]:
team1 = teams_df[teams_df[Team_Name].str.find('North Carolina') >= 0][Team_Id].values[0]
team2 = teams_df[teams_df[Team_Name].str.find('Villanova') >= 0][Team_Id].values[0]
team1, team2
diff = teams_vectors[team1] - teams_vectors[team2]
diff['Wloc'] = 0
d(diff)

model = clf_LR
model.fit(X_train, y_train)
p(model.predict(diff)[0])
model.predict_proba(diff)

# Predict the results of the season

In [ ]:
def predict_table(source, both=True):
  result_df = pd.DataFrame()
  for index, row in source.iterrows():
    teamW = row[Wteam]
    teamL = row[Lteam] 

    diff = teams_vectors[teamW] - teams_vectors[teamL]
    diff['Wloc'] = 1 if row[Wloc] == 'H' else 0 if row[Wloc] == 'N' else -1

    result = pd.DataFrame(model.predict_proba(diff))

    result['Teams'] = f'{teamW}_{teamL}'
    result['Correct'] = 1 if result.iloc[0,1] > 0.5 else 0
    result_df = result_df.append(result)
    if not both:
      continue
    diff = -diff
    result = pd.DataFrame(model.predict_proba(diff))
    result['Teams'] = f'{teamL}_{teamW}'
    result_df = result_df.append(result)
  return result_df

In [ ]:
result_df = predict_table(season)

In [ ]:
result_df.shape

In [ ]:
pd.set_option("precision", 5)
p(result_df.mean())
pd.set_option("precision", 2)

# Results of tourney for year



In [ ]:
year

In [ ]:
tourney_year = tourneyCompactResults_df[tourneyCompactResults_df.Season == year]
d(tourney_year.head())
tourney_result_df = predict_table(tourney_year, False)
tourney_result_df['Actual'] = 1
d(tourney_result_df.head())
p(tourney_result_df.Correct.sum() / len(tourney_result_df))
p(tourney_result_df.Correct.sum())

In [ ]:
from sklearn.metrics import log_loss

# since log_loss does not accept y_true that all values are 1, we artificially switch row0 proba and set the y to 0
#p('Test logloss is {:.5f}'.format(log_loss(tourney_result_df[[0,1]], tourney_result_df.Correct)))
y_true = list(tourney_result_df.Actual.values)
y_true[0]=0
y_pred = tourney_result_df[[0,1]].copy()
x0 = y_pred.iloc[0,0]
x1 = y_pred.iloc[0,1]
y_pred.iloc[0,0], y_pred.iloc[0,1] = x1, x0
p(log_loss(y_true, y_pred))

# Grid Search for hyperparameters - on best model

In [ ]:
GS = False
if GS:
  clf_LR.fit(X_train, y_train)
  p(clf_LR.score(X_train, y_train))
  p(clf_LR.score(X_test, y_test))

In [ ]:
if GS:
  param_grid = { 
      'C' : [1, 10, 100, 1000],
      'warm_start': [True],
      'max_iter': [1000, 2000, 5000],
      'fit_intercept': [False],
  }

  clf_gs = GridSearchCV(clf_LR, param_grid, cv=5, verbose=4)
  clf_gs.fit(X_train, y_train)

In [ ]:
if GS:
  p('Results')
  clf_gs.estimator
  p(clf_gs.best_params_)
  p(clf_gs.best_estimator_)

  p(clf_gs.score(X_train, y_train))
  p(clf_gs.score(X_test, y_test))

# TODO - all done

> 1. normalize columns - DONE, no improvement

> 2. add column - win * days - the latest the win the better - DONE, windays - no improvement, lossdays - worse

> 3. add column - wins * seed - wins agains higher ranking teams - better -DONE, improved from 43 to 45 correct predictions, and when we added also losses * seed with a -ve sign, the results improved to 47 correct predictions (70%)

> 4. add games for prev season - so that 2 seasons are taken into account - WOW, improvement of 2 more correct results to 49 (73%)

> 5. Cross_val

> 6. ROC curve

# NBA

## get data from web for team=CLE - Cleveland Cavaliers

In [2]:
import urllib.request
import re
import requests
from bs4 import BeautifulSoup as bs
import json
import numpy as np
import pandas as pd
from datetime import datetime

try:
  from requests_html import HTMLSession, AsyncHTMLSession
except:
  !pip install requests-html  
  from requests_html import HTMLSession, AsyncHTMLSession

from IPython.display import display
p = print
d = display  

     |████████████████████████████████| 92kB 4.4MB/s 
     |████████████████████████████████| 81kB 8.1MB/s 
     |████████████████████████████████| 81kB 7.8MB/s 
  Created wheel for fake-useragent: filename=fake_useragent-0.1.11-cp37-none-any.whl size=13485 sha256=da2d09422e4329cc8d404fb9a49c43d1a53d3340fab792c575fafad6e5bb83d9
  Stored in directory: /root/.cache/pip/wheels/5e/63/09/d1dc15179f175357d3f5c00cbffbac37f9e8690d80545143ff
  Created wheel for parse: filename=parse-1.19.0-cp37-none-any.whl size=24581 sha256=286aa249dad0262411267d156a9804608b45b2e2c53e812702679e23770499eb
  Stored in directory: /root/.cache/pip/wheels/c0/39/ea/e2fd678bd130953f5438470b8dfa529f00787e9b8b92b27467
Successfully built fake-useragent parse
ERROR: pyppeteer 0.2.5 has requirement urllib3<2.0.0,>=1.25.8, but you'll have urllib3 1.24.3 which is incompatible.
  Found existing installation: importlib-metadata 4.0.1
    Uninstalling importlib-metadata-4.0.1:
      Successfully uninstalled importlib-metadata-

In [3]:
def get_soup(url):
  resp = requests.get(url, headers={'User-Agent': 'Mozilla/5.0'})
  if resp.status_code == 200:
    soup = bs(resp.text, 'html.parser')
    return soup, resp.status_code
  print('error accesing site code:', resp.status_code)
  return None, None

In [4]:
!pip install selenium 

     |████████████████████████████████| 911kB 5.3MB/s 


In [5]:
from bs4 import BeautifulSoup as bs
from time import sleep
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
!apt-get update 
!apt install chromium-chromedriver

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:6 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Ign:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:9 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:10 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:11 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [56.8 kB]
Hit:12 http://ppa.launchpad.net/cran/

In [6]:
#soup, ret_code = get_soup('https://www.basketball-reference.com/teams/CLE/2016/gamelog/')

In [145]:
chrome_options = Options()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

def get_nba(year, cur_team):
  wd = webdriver.Chrome('chromedriver',options=chrome_options)
  driver =webdriver.Chrome('chromedriver',options=chrome_options)
  URL = "https://www.basketball-reference.com/teams/CLE/2016/gamelog/"
  URL = f'https://www.basketball-reference.com/teams/{cur_team}/{year}/gamelog/'

  driver.get(URL)
  sleep(5) # give time for all javascripts to be finished running
  page0 = driver.page_source
  driver.close()

  soup = bs(page0, "lxml")

    # save to disk so to load the HTML to a txt file - easier to look at because in Ju[pyter it is too slow]
  with open(f'NBA{year}.html', 'w') as f:
    f.write(soup.prettify())
  del page0

  #with open('corona.txt') as f:
  #  soup = f.read()    
  return soup

In [36]:
def get_name(soup):
  team_name = soup.findAll('title')[0].get_text()[8:] # skip season 2014-15
  idx = team_name.index(' Team')
  team_name = team_name[:idx]
  return team_name

In [151]:
def save_team(df):
  df_teams = pd.DataFrame.from_dict(teams, orient='index', columns=['TeamName'])
  df_teams.index.name = 'TeamID'
  df_teams.reset_index(inplace=True)
  df_teams.to_csv(f'teams.csv', index=False)
  p('Teams:', df_teams.shape)
  d(df_teams.head())

In [115]:
def get_next_team(teams):
  cur_team = ''
  for k, v in teams.items():
    if v == '':
      cur_team = k
      break
  return cur_team


regularSeasonDetailedResults_df.columns

Index(['Season', 'DayNum', 'WTeamID', 'WScore', 'LTeamID', 'LScore', 'WLoc',
       'NumOT', 'WFGM', 'WFGA', 'WFGM3', 'WFGA3', 'WFTM', 'WFTA', 'WOR', 'WDR',
       'WAst', 'WTO', 'WStl', 'WBlk', 'WPF', 'LFGM', 'LFGA', 'LFGM3', 'LFGA3',
       'LFTM', 'LFTA', 'LOR', 'LDR', 'LAst', 'LTO', 'LStl', 'LBlk', 'LPF'],
      dtype='object')

WFGM - field goals made (by the winning team) - FG

WFGA - field goals attempted (by the winning team) - FGA

WFGM3 - three pointers made (by the winning team) - 3P

WFGA3 - three pointers attempted (by the winning team) - 3PA
 WFTM - free throws made (by the winning team) WFTA - free throws attempted (by the winning team) WOR - offensive rebounds (pulled by the winning team) WDR - defensive rebounds (pulled by the winning team) WAst - assists (by the winning team) WTO - turnovers committed (by the winning team) WStl - steals (accomplished by the winning team) WBlk - blocks (accomplished by the winning team) WPF - personal fouls committed (by the winning team)



In [114]:
def set_nba_data(row, cur_team):
  if row['W/L'] == 'W':
    wteam = cur_team
    lteam = row.OppTeam
    wscore = row.Tm
    lscore = row.Opp
    wloc = 'A' if row.Home == '@' else 'H'
    x = 0
    y = 1
  else:
    wteam = row.OppTeam
    lteam = cur_team
    wscore = row.Opp
    lscore = row.Tm
    wloc = 'H' if row.Home == '@' else 'A'
    x = 1
    y = 0

  WFGM = row.FG.values[x]
  WFGA = row.FGA.values[x]
  WFGM3 = row['3P'].values[x]
  WFGA3 = row['3PA'].values[x]
  WFTM = row.FT.values[x]
  WFTA = row.FTA.values[x]
  WOR = row.ORB.values[x]
  WDR = row.TRB.values[x]
  WAst = row.AST.values[x]
  WTO = row.TOV.values[x]
  WStl = row.STL.values[x]
  WBlk = row.BLK.values[x]
  WPF = row.PF.values[x]

  LFGM = row.FG.values[y]
  LFGA = row.FGA.values[y]
  LFGM3 = row['3P'].values[y]
  LFGA3 = row['3PA'].values[y]
  LFTM = row.FT.values[y]
  LFTA = row.FTA.values[y]
  LOR = row.ORB.values[y]
  LDR = row.TRB.values[y]
  LAst = row.AST.values[y]
  LTO = row.TOV.values[y]
  LStl = row.STL.values[y]
  LBlk = row.BLK.values[y]
  LPF = row.PF.values[y]

  return wteam, wscore, lteam, lscore, wloc, WFGM, WFGA, WFGM3, WFGA3, WFTM, WFTA, WOR, WDR,\
       WAst, WTO, WStl, WBlk, WPF, LFGM, LFGA, LFGM3, LFGA3,\
       LFTM, LFTA, LOR, LDR, LAst, LTO, LStl, LBlk, LPF

In [ ]:
df_reg = pd.DataFrame()
df_off = df_reg.copy()
cur_team = 'CLE'
teams = {cur_team:''}  # starting team

while True:
  for year in range(2015, 2021):
    p('\ndoing:', year, cur_team, end='')
    soup = get_nba(year, cur_team)
    divs = soup.find_all('div', class_="table_wrapper")
    p(', ndivs:', len(divs), end='')
#    for div in divs:
#      p(year, div.find('h2').text )
    for k in range(len(divs)):
      table = divs[k].findAll('table')[0]  # one table only
      rows = table.findAll('tr')
      ncols = 0
      columns = []
      lines = []
      for i, row in enumerate(rows):
      #  p(f'[{i}] {row}')
        if i < 2:
          cols = row.findAll(['th'])
        else:
          cols = row.findAll(['th', 'td'])
        line = []
        for j, col in enumerate(cols):
          txt = col.get_text().strip()
          #if len(txt) > 0:      p(txt)
          if i > 1 and j == 0 and (len(txt) == 0 or txt == 'Rk'):
            break    # skip headers between tables
          if i == 0:  # i=0 size of each group of cols = total is size of row
            try:
              ncol = int(col["colspan"])
              ncols += ncol
      #        p(ncols, ncol)
            except:
              pass
          elif i == 1:
            columns.append(txt)
          else:
            line.append(txt)
        if i > 1 and len(line) > 0:
          lines.append(line)
      df = pd.DataFrame(lines, columns=columns)
      df.drop(['Rk', 'G'], axis=1, inplace=True)
      new_columns = df.columns.values;  new_columns[1] = 'Home'; new_columns[2] = 'OppTeam'; df.columns  = new_columns
      df['Season'] = year
      first_col = df.pop('Season')
      df.insert(0, 'Season', first_col)
      
      df_r = pd.DataFrame()
      df_r['Season'] = df['Season']
      df_r['Date'] = df.Date
      df_r['WTeamID'], df_r['WScore'], df_r['LTeamID'], df_r['LScore'], df_r['WLoc'],\
            df_r['WFGM'], df_r['WFGA'], df_r['WFGM3'], df_r['WFGA3'], df_r['WFTM'], df_r['WFTA'], df_r['WOR'], df_r['WDR'],\
            df_r['WAst'], df_r['WTO'], df_r['WStl'], df_r['WBlk'], df_r['WPF'], df_r['LFGM'], df_r['LFGA'], df_r['LFGM3'], df_r['LFGA3'],\
            df_r['LFTM'], df_r['LFTA'], df_r['LOR'], df_r['LDR'], df_r['LAst'], df_r['LTO'], df_r['LStl'], df_r['LBlk'], df_r['LPF'] \
          =  zip(*df.apply(set_nba_data, cur_team=cur_team, axis=1))

      if k == 0:
        fname = 'regular'
        df_reg = df_reg.append(df_r)
      else:
        fname = 'playoff'
        df_off = df_off.append(df_r) 
      p(f', shape {fname}:', df_r.shape, end='')

      # get teams - in case a new team was added in a season
      teams_list = list(df.OppTeam.values)
      for t in teams_list:
        teams[t] = teams.get(t, '')
      teams[cur_team] = get_name(soup)
    # get next team - one that does not have a name
  cur_team = get_next_team(teams)
  if cur_team == '':
    break
  p()

save_team(teams)

p('Before drop_dup reg:', df_reg.shape)
df_reg.drop_duplicates(inplace=True)
df_reg.to_csv(f'regular.csv', index=False)  
p('After drop_dup reg:', df_reg.shape)

p('Before drop_dup off:', df_off.shape)
df_off.drop_duplicates(inplace=True)
df_off.to_csv(f'playoff.csv', index=False)    
p('After drop_dup off:', df_off.shape)


doing: 2015 CLE, ndivs: 2, shape regular: (82, 33), shape playoff: (20, 33)
doing: 2016 CLE

In [142]:
p(df_off.shape)
mask = ((df_off.Date == '2015-04-19') & (df_off.WTeamID=='CLE'))
p(mask.sum())
df_off[mask]

(162, 33)
2


,Season,Date,WTeamID,WScore,LTeamID,LScore,WLoc,WFGM,WFGA,WFGM3,WFGA3,WFTM,WFTA,WOR,WDR,WAst,WTO,WStl,WBlk,WPF,LFGM,LFGA,LFGM3,LFGA3,LFTM,LFTA,LOR,LDR,LAst,LTO,LStl,LBlk,LPF
0,2015,2015-04-19,CLE,113,BOS,100,H,37,82,13,31,26,33,15,46,19,12,6,5,23,37,79,8,22,18,22,7,34,24,14,7,4,29
0,2015,2015-04-19,CLE,113,BOS,100,H,37,82,13,31,26,33,15,46,19,12,6,5,23,37,79,8,22,18,22,7,34,24,14,7,4,29


In [143]:
df_nodups = df_off.drop_duplicates()
p(df_nodups.shape)
mask = ((df_nodups.Date == '2015-04-19') & (df_nodups.WTeamID=='CLE'))
p(mask.sum())
df_nodups[mask]

(81, 33)
1


,Season,Date,WTeamID,WScore,LTeamID,LScore,WLoc,WFGM,WFGA,WFGM3,WFGA3,WFTM,WFTA,WOR,WDR,WAst,WTO,WStl,WBlk,WPF,LFGM,LFGA,LFGM3,LFGA3,LFTM,LFTA,LOR,LDR,LAst,LTO,LStl,LBlk,LPF
0,2015,2015-04-19,CLE,113,BOS,100,H,37,82,13,31,26,33,15,46,19,12,6,5,23,37,79,8,22,18,22,7,34,24,14,7,4,29


In [131]:
d(df.head(3))
d(df_reg.head(3))

,Season,Date,Home,OppTeam,W/L,Tm,Opp,FG,FGA,FG%,3P,3PA,3P%,FT,FTA,FT%,ORB,TRB,AST,STL,BLK,TOV,PF,,FG,FGA,FG%,3P,3PA,3P%,FT,FTA,FT%,ORB,TRB,AST,STL,BLK,TOV,PF
0,2015,2014-10-30,,NYK,L,90,95,32,70,.457,9,24,.375,17,20,.850,11,33,22,5,2,19,15,,37,69,.536,6,12,.500,15,18,.833,9,35,30,9,1,15,26
1,2015,2014-10-31,@,CHI,W,114,108,41,104,.394,3,15,.200,29,33,.879,20,52,18,8,4,9,18,,41,90,.456,13,31,.419,13,17,.765,11,42,25,4,14,19,28
2,2015,2014-11-04,@,POR,L,82,101,31,85,.365,9,21,.429,11,14,.786,13,41,18,5,3,14,19,,37,75,.493,8,25,.320,19,25,.760,9,48,21,7,4,11,20


,Season,Date,WTeamID,WScore,LTeamID,LScore,WLoc,WFGM,WFGA,WFGM3,WFGA3,WFTM,WFTA,WOR,WDR,WAst,WTO,WStl,WBlk,WPF,LFGM,LFGA,LFGM3,LFGA3,LFTM,LFTA,LOR,LDR,LAst,LTO,LStl,LBlk,LPF
0,2015,2014-10-30,NYK,95,CLE,90,A,37,69,6,12,15,18,9,35,30,15,9,1,26,32,70,9,24,17,20,11,33,22,19,5,2,15
1,2015,2014-10-31,CLE,114,CHI,108,A,41,104,3,15,29,33,20,52,18,9,8,4,18,41,90,13,31,13,17,11,42,25,19,4,14,28
2,2015,2014-11-04,POR,101,CLE,82,H,37,75,8,25,19,25,9,48,21,11,7,4,20,31,85,9,21,11,14,13,41,18,14,5,3,19


# END